# New-RscMssqlSnapshot
DBAs will often take a backup before they make a change to the database. In general, you need 2 things to take a snapshot of a database, the database ID and the SLA ID. An SLA is required for all on demand snapshots to manage the retention of the snapshot. 

# Workflow
To start an On-Demand Snapsot of a database, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscHost** to get the id of the host. This will be used in the next cmdlet when looking up a SQL Instance
1. Use **Get-RscMssqlInstance** to get the instance id of the SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Get-RscMssqlDatabase** to get the database ID based on the database name and the SQL Server Instance. 
1. Use **Get-RscSLADomain** to get the ID SLA Domain that will manage the retention of the snapshot.
1. Use **New-RscMssqlSnapshot** to initiate an On Demand Snapshot of a database. 
1. Use **Get-RscRequest** to track the request and know when it completes. ***DOES NOT EXIST YET***
1. Use **Disconnect-RSC** to disconnect from RSC

## Connect to RSC

In [ ]:
Import-Module "/Users/Chris.Lumnah/Library/CloudStorage/GoogleDrive-chris.lumnah@rubrik.com/My Drive/1.Projects/Powershell SDK/rubrik-powershell-sdk/output/RubrikSecurityCloud"
# Import-Module RubrikSecurityCloud
Connect-RSC

## Get the Cluster ID

In [ ]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName

## Get the Host ID

In [ ]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows

## Get the Instance Information

In [ ]:
$RscMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster

## Get the Database Information

In [ ]:
$DatabaseName = "AdventureWorks2019"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName  -RscMssqlInstance $RscMssqlInstance

## Get SLA Domain Information

In [ ]:
$SLADomain = "MSSQL-1d-30d-NoArchive-Repl-Houghton"
$RscSlaDomain = Get-RscSlaDomain -Name $SLADomain
$RscSlaDomain

## Initiate On Demand Snapshot

In [ ]:
New-RscMssqlSnapshot -RscMssqlDatabase $RscMssqlDatabase -RscSlaDomain $RscSlaDomain

## Track the status of the backup

---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [ ]:
Disconnect-Rsc